In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine 

In [20]:
#Read in the Crime excel file
crime_file = "Crime Rates by Metropolitan Areas 2016.xls"
crime_df = pd.read_excel(crime_file)
crime_df.head()

,"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.",Note that not all areas show Violent Crime Totals.,Population,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.","Rate per 100,000 inhabitants",2157729,245.3,23.4,4.9,123.0,94,1190.4,255.6,778.7,156.1
1,"Fairbanks, AK M.S.A.","Rate per 100,000 inhabitants",34657,669.4,20.2,101.0,92.3,455.9,4377.2,499.2,3482.7,395.3
2,"Detroit-Dearborn-Livonia, MI M.D.","Rate per 100,000 inhabitants",1747582,1034.2,19.4,63.2,220.9,730.7,3132.6,735.0,1725.4,672.1
3,"New Orleans-Metairie, LA M.S.A.","Rate per 100,000 inhabitants",1272995,558.8,18.9,56.0,162.0,321.9,3001.3,468.3,2245.5,287.5
4,"Memphis, TN-MS-AR M.S.A.","Rate per 100,000 inhabitants",1346931,1082.1,18.5,54.6,269.4,739.6,3868.2,934.3,2623.2,310.6


In [38]:
#Transform Crime Data Frame 
crime_cols = crime_df[["Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.","Aggravated\nassault", "Violent\ncrime","Murder and\nnonnegligent\nmanslaughter", "Robbery"]]
crime_cols.head()


,"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.",Aggravated assault,Violent crime,Murder and nonnegligent manslaughter,Robbery
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4


In [63]:
#Rename the Columns
crime_clean = crime_cols.rename(columns={"Metropolitan Statistical Area; places in light type are parts of bigger metro areas, in bold.": "Location",
                                        "Aggravated\nassault": "Aggravated Assault",
                                        "Murder and\nnonnegligent\nmanslaughter": "Murder",
                                        "Violent\ncrime": "Violent Crime"})
crime_clean.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4


In [64]:
#Drop empty data
crime_clean.dropna(inplace = True)
crime_clean=pd.concat([crime_clean,crime_clean.Location.str.split(',',expand=True)],1)

crime_clean.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,0,1,2
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,None


In [65]:
#Rename the new columns
crime_split = crime_clean.rename(columns={0: "City1",
                                        1: "State"})
crime_split.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State,2
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,None


In [67]:
#Drop the columns I dont need
crime_split = crime_split[["Location", "Aggravated Assault", "Violent Crime", "Murder", "Robbery", "City1", "State"]]
crime_split.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.


In [68]:
#Strip down the city names 
crime_split2=pd.concat([crime_split,crime_split.City1.str.split('-',expand=True)],1)
crime_split2.head()

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City1,State,0,1,2,3
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan-Carolina-Caguas,Puerto Rico M.S.A.,San Juan,Carolina,Caguas,None
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.,Fairbanks,None,None,None
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit-Dearborn-Livonia,MI M.D.,Detroit,Dearborn,Livonia,None
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans-Metairie,LA M.S.A.,New Orleans,Metairie,None,None
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.,Memphis,None,None,None


In [70]:
#Clean up the final table!!! 
crime_split2 = crime_split2.rename(columns={0: "City"})

crime_final = crime_split2[["Location", "Aggravated Assault", "Violent Crime", "Murder", "Robbery", "City", "State"]]
crime_final.head()                            

,Location,Aggravated Assault,Violent Crime,Murder,Robbery,City,State
0,"San Juan-Carolina-Caguas, Puerto Rico M.S.A.",94,245.3,23.4,123.0,San Juan,Puerto Rico M.S.A.
1,"Fairbanks, AK M.S.A.",455.9,669.4,20.2,92.3,Fairbanks,AK M.S.A.
2,"Detroit-Dearborn-Livonia, MI M.D.",730.7,1034.2,19.4,220.9,Detroit,MI M.D.
3,"New Orleans-Metairie, LA M.S.A.",321.9,558.8,18.9,162.0,New Orleans,LA M.S.A.
4,"Memphis, TN-MS-AR M.S.A.",739.6,1082.1,18.5,269.4,Memphis,TN-MS-AR M.S.A.
